In [ ]:
###############################################################################################
## chap 2. plotly 시각화 만들기
###############################################################################################

In [ ]:
## 필요한 라이브러리 로딩
import pandas as pd
from datetime import datetime, timedelta
from pandas.api.types import CategoricalDtype
pd.set_option('mode.chained_assignment',  None)

## 1. covid19 원본 데이터셋 로딩
## covid19 데이터 로딩(파일을 다운로드 받은 경우)
df_covid19 = pd.read_csv("D:/R/data/Rnpy//owid-covid-data.csv")

## covid19 데이터 로딩(온라인에서 바로 로딩할 경우)
## df_covid19 = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
## 2. 전체 데이터셋 중 최근 100일간의 데이터를 필터링한 df_covid19_100 생성
## df_covid19['date']를 datetime으로 변환
df_covid19['date'] = pd.to_datetime(df_covid19['date'], format="%Y-%m-%d")

## 대륙 데이터와 최종 데이터로부터 100일 전 데이터 필터링
df_covid19_100 = df_covid19[(df_covid19['iso_code'].isin(['KOR', 'OWID_ASI', 'OWID_EUR', 
'OWID_OCE', 'OWID_NAM', 'OWID_SAM', 'OWID_AFR'])) 
& (df_covid19['date'] >= (max(df_covid19['date']) - timedelta(days = 100)))]

## 대륙명을 한글로 변환
df_covid19_100.loc[df_covid19_100['location'] == 'South Korea', "location"] = '한국'
df_covid19_100.loc[df_covid19_100['location'] == 'Asia', "location"] = '아시아'
df_covid19_100.loc[df_covid19_100['location'] == 'Europe', "location"] = '유럽'
df_covid19_100.loc[df_covid19_100['location'] == 'Oceania', "location"] = '오세아니아'
df_covid19_100.loc[df_covid19_100['location'] == 'North America', "location"] = '북미'
df_covid19_100.loc[df_covid19_100['location'] == 'South America', "location"] = '남미'
df_covid19_100.loc[df_covid19_100['location'] == 'Africa', "location"] = '아프리카'

## 이산형 변수 설정
ord = CategoricalDtype(categories = ['한국', '아시아', '유럽', '북미', '남미', 
'아프리카','오세아니아'], ordered = True)

df_covid19_100['location'] = df_covid19_100['location'].astype(ord)

## date로 정렬
df_covid19_100 = df_covid19_100.sort_values(by = 'date')

## 3. df_covid19_100을 한국과 각 대륙별 열로 배치한 넓은 형태의 데이터프레임으로 변환
df_covid19_100_wide = df_covid19_100.loc[:,['date', 'location', 'new_cases', 
'people_fully_vaccinated_per_hundred']].rename(columns={'new_cases':'확진자', 
'people_fully_vaccinated_per_hundred':'백신접종완료자'})

df_covid19_100_wide = df_covid19_100_wide.pivot(index='date', columns='location', 
values=['확진자', '백신접종완료자']).sort_values(by = 'date')

df_covid19_100_wide.columns = ['확진자_한국', '확진자_아시아', '확진자_유럽', '확진자_북미',  
'확진자_남미', '확진자_아프리카', '확진자_오세아니아', '백신접종완료자_한국', 
'백신접종완료자_아시아', '백신접종완료자_유럽', '백신접종완료자_북미', 
'백신접종완료자_남미', '백신접종완료자_아프리카', '백신접종완료자_오세아니아']

## 4. covid19 데이터를 국가별로 요약한 df_covid19_stat 생성
df_covid19_stat = df_covid19.groupby(['iso_code', 'continent', 'location'], dropna=True).agg(
    인구수 = ('population', 'max'), 
    전체사망자수 = ('new_deaths', 'sum'), 
    백신접종자완료자수 = ('people_fully_vaccinated', 'max'), 
    인구백명당백신접종완료율 = ('people_fully_vaccinated_per_hundred', 'max'), 
    인구백명당부스터접종자수 = ('total_boosters_per_hundred', 'max')).reset_index()

df_covid19_stat['십만명당사망자수'] = round(df_covid19_stat['전체사망자수'] / 
                                    df_covid19_stat['인구수'] * 100000, 5)

df_covid19_stat['백신접종완료율'] = df_covid19_stat['백신접종자완료자수'] / \
df_covid19_stat['인구수']

## 여백 설정을 위한 변수 설정
margins_P = {'t' : 50, 'b' : 25, 'l' : 25, 'r' : 25}

In [ ]:
## 파이썬 코드
## 대학 학과 취업률 데이터 로딩
df_취업률 = pd.read_excel("D:/R/data/Rnpy/2021년 학과별 고등교육기관 취업통계.xlsx", \
                       ## '학과별' 시트의 데이터를 불러오는데,
                       sheet_name = '학과별', \
                       ## 앞의 13행을 제외하고 \
                       skiprows=(13), \
                       ## 첫 번째 행은 열 이름으로 설정
                       header = 0)

## df_취업률에서 첫 번째부터 9번째까지의 열과 '계'로 끝나는 열을 선택하여 다시 df_취업률에 저장
df_취업률 = pd.concat([df_취업률.iloc[:, 0:8],
                    df_취업률.loc[:, df_취업률.columns.str.endswith('계')], \
                    df_취업률.loc[:, '입대자']], \
                    axis = 1)

## df_취업률에서 졸업자가 500명 이하인 학과 중 25% 샘플링
df_취업률_500 = df_취업률.loc[(df_취업률['졸업자_계'] < 500)]
df_취업률_500 = df_취업률_500.iloc[range(0, len(df_취업률_500.index) , 4)]
df_취업률_500 = df_취업률_500.rename(columns = {'졸업자_계':'졸업자수', '취업률_계': 
'취업률', '취업자_합계_계':'취업자수'})

In [ ]:
## 파이썬에서 Plotly 라이브러리 모듈 로딩
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
## 파이썬에서 Plotly 객체 초기화
import plotly.graph_objects as go
fig = go.Figure()
fig.show()

In [ ]:
## Plotly 초기화 함수로 data 속성 값 설정
go.Figure(
    ## data 키워드로 data 속성의 설정임을 명시
    data = [{
        ## scatter 트레이스의 markers 속성 설정
        'type' : 'scatter', 'mode' : 'markers',
        ## X, Y 축에 변수 매핑
        'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'new_cases'],
        ## 마커 색상 설정
        'marker' : {'color' : '#264E86'}
        },
        ## scatter 트레이스의 lines 속성 설정
        {'type' : 'scatter', 'mode' : 'lines',
        ## X, Y 축에 변수 매핑
        'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'new_cases'],
        ## 라인 색상과 대시 설정
        'line' : {'color' : '#5E88FC', 'dash' : 'dash'}
        }
    ]).show()

In [ ]:
fig = go.Figure(
data = {
    'type' : 'scatter',
    'mode' : 'markers+lines',
    'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'date'],
    'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'new_cases'],
    'marker' : {'color' : '#264E86'},
    'line' : {'color' : '#5E88FC', 'dash' : 'dash'}
    }, 
    layout = {                                           ## layout 속성의 설정
        'title' : "코로나19 발생 현황",                    ## 전체 제목 설정
        'xaxis' : {'title' : "날짜", 'showgrid' : False}, ## X축 layout 속성 설정
        'yaxis' : {'title' : "확진자수"},                 ## y축 layout 속성 설정
        'margin' : margins_P                              ## 여백 설정
        })

fig.show()

In [ ]:
fig = go.Figure(
data = [
    {
        'type' : 'scatter',
        'mode' : 'markers+lines',
        'x' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'date'],
        'y' : df_covid19_100.loc[df_covid19_100['iso_code'] == 'KOR', 'new_cases'],
        'marker' : {'color' : '#264E86'},
        'line' : {'color' : '#5E88FC', 'dash' : 'dash'}
    }])
    
print(fig)

In [ ]:
###############################################################################################
## chap 3. 트레이스
###############################################################################################

In [ ]:
## Plotly 초기화
fig = go.Figure()

fig.add_trace({         ## 트레이스 추가
    'type' : 'scatter', ## scatter 트레이스
    'mode' : 'markers', ## markers 모드
    'x' : df_취업률_500['졸업자수'], 'y' : df_취업률_500['취업자수'],
    'marker' : { 'size' : 3, 'color' : 'darkblue'} ## marker 크기와 색상 설정
    })

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
  { ## data 속성들로 구성되 딕셔너리 정의
    'mode' : 'markers',
    'x' : df_취업률_500['졸업자수'], 'y' : df_취업률_500['취업자수'],
    'marker' : {'size' : 3, 'color' : 'darkblue'}
  } ## {} 사용
))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter( ## 매개변수 형태의 속성 할당 방법 사용
    mode = 'markers',
    x = df_취업률_500['졸업자수'], y = df_취업률_500['취업자수'],
    marker = dict(size = 3, color = 'darkblue') ## dict() 사용
))

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], ## X, Y 축에 매핑되는 변수 설정
    'y': df_취업률_500['취업자수']})

fig.show()


In [ ]:
fig = go.Figure()

for 대계열, group in df_취업률_500.groupby('대계열'):
    fig.add_trace({
        'type' : 'scatter', 'mode' : 'markers',
        'x': group['졸업자수'], 'y': group['취업자수'],
        'name' : 대계열}) ## name 속성 설정

fig.show()

In [ ]:
########################
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    ## marker 외부에서 opacity 설정
    'opacity' : 0.3}) ## opacity를 0.3으로 설정

fig.show()


In [ ]:
########################
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    ## marker 내부에서 opacity 설정
    'marker' : {'opacity' : 0.3}}) ## opacity를 0.3으로 설정
    
fig.show()

In [ ]:
fig = go.Figure()

for 대계열, group in df_취업률_500.groupby('대계열'):
    fig.add_trace({
        'type' : 'scatter', 'mode' : 'markers',
        'x': group['졸업자수'], 'y': group['취업자수'],
        'name' : 대계열,
        'showlegend' : False}) ## showlegend를 FALSE로 설정
        
fig.show()

In [ ]:
fig = go.Figure()

## 긴 형태의 100일간 코로나19 데이터를 국가명으로 그룹화, 확진자수의 합계를 new_cases로산출하여 temp에 저장
temp = df_covid19_100.groupby('location').agg(new_cases = ('new_cases', 'sum'))

fig.add_trace({
    'type' : 'bar', ## bar 트레이스 설정
    'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'] ## 텍스트 설정
})

fig.show()

In [ ]:
temp = df_covid19_100.groupby('location').agg(new_cases = ('new_cases', 'sum'))

#######################################
fig = go.Figure()

fig.add_trace({
    'type' : 'bar', 'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'],
    'textposition' : 'inside'}) ## textposition을 'inside'로 설정

fig.show()

#######################################
fig = go.Figure()

fig.add_trace({
    'type' : 'bar', 'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'],
    'textposition' : 'outside'}) ## textposition을 'outside'로 설정

fig.show()

#######################################
fig = go.Figure()

fig.add_trace({
    'type' : 'bar', 'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'],
    'textposition' : 'auto'}) ## textposition을 'auto'로 설정

fig.show()

#######################################
fig = go.Figure()

fig.add_trace({
    'type' : 'bar', 'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'],
    'textposition' : 'none'}) ## textposition을 'none'으로 설정

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'bar', 'x': temp.index, 'y': temp['new_cases'],
    'text' : temp['new_cases'], 'textposition' : 'inside',
    'texttemplate' : '확진자수:%{text:,}'}) ## texttemplate를 설정

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    'hoverinfo' : 'y'}) ## hoverinfo 설정

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    ## hovertext의 설정
    'hovertext' : '중계열:'+ df_취업률_500['중계열']+'<br>'+'소계열:'+df_취업률_500['소계열']})

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    'hovertext' : df_취업률_500['중계열'],
    ## hovertemplate의 설정
    'hovertemplate' : ' 졸업자:%{x}, 취업자:%{y}, 대계열:%{hovertext}'})

fig.show()

In [ ]:
###############################################################################################
##  chap 4. layout 속성
###############################################################################################

In [ ]:
fig_scatter = go.Figure()

fig_scatter.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x' : df_취업률_500['졸업자수'], 'y' : df_취업률_500['취업자수'], 'marker' : {'color' : 'darkblue'}})

fig_scatter.update_layout(
    ## dict()를 사용한 title 속성의 설정
    title = dict(text = "<b>졸업자 대비 취업자수</b>",
    x = 0.5, xanchor = 'center', yanchor = 'top'))

fig_scatter.show()

In [ ]:
fig_scatter_temp = go.Figure(fig_scatter)

## title의 HTML inline 설정
fig_scatter_temp.update_layout(
title = {'text' : "<span style = 'font-size:15pt'> \
<span style = 'color:red;font-weight:bold;'> 졸업자</span> \
<span style = 'font-size:10pt'> 대비</span> \
<span style = 'color:blue;font-weight:bold;'>취업률</span></span>", 
'x' : 0.5, 'xanchor' : 'center', 'yanchor' : 'top'})

fig_scatter_temp.show()

In [ ]:
  ## 페이퍼 배경색과 플롯 배경색의 설정
fig_scatter.update_layout(paper_bgcolor = 'lightgray', plot_bgcolor = 'lightgray')
fig_scatter.show()

In [ ]:
fig_scatter.update_layout(
    xaxis = dict(
        title = dict(text = '<b>학과 졸업자수</b><sub>(명)</sub>'), ## 정상적 방법
        color = 'black', zerolinecolor = 'black', zerolinewidth = 3,
        gridcolor = 'gray', gridwidth = 1),
        yaxis = dict(
            title = '<b>학과 취업자수</b><sub>(명)</sub>', ## 약식 방법
            color = 'black', zerolinecolor = 'black', zerolinewidth = 3,
            gridcolor = 'gray', gridwidth = 1))

fig_scatter.show()

In [ ]:
fig_scatter.update_layout(
    xaxis = dict(tickmode = 'array',                     ## tickmode를 "array"로 설정 
                ticktext = ('소규모', '중규모', '대규모'), ## ticktext 설정
                tickvals = (100, 300, 400)               ## tickvals 설정
                 ),
    yaxis = dict(tickmode = 'linear', ## tickmode를 "linear"로 설정
                tick0 = 100,          ## tick0 설정
                dtick = 100)          ## dtick 설정
)

sfig_scatter.show()

In [ ]:
fig_scatter = go.Figure(fig_scatter)
fig_scatter.update_layout(
    xaxis = dict(range = (0, 350), ## 튜플을 사용한 X축의 range 설정 
                rangemode = 'nonnegative'), ## X축의 rangemode 설정
    yaxis = dict(range = [0, 300], ## 리스트를 사용한 Y축의 range 설정
                rangemode = 'tozero'), ## Y축의 rangemode 설정
    margin = dict(pad = 5)
    )

fig_scatter.show()

In [ ]:
## 초기화
fig_line = go.Figure()

## 한국 확진자 선 scatter 트레이스 생성
fig_line.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x' : df_covid19_100_wide.index, 'y' : df_covid19_100_wide['확진자_한국'],
    'name' : '한국'
})

## 아시아 확진자 선 scatter 트레이스 추가
fig_line.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x' : df_covid19_100_wide.index, 'y' : df_covid19_100_wide['확진자_아시아'],
    'name' : '아시아' , 'showlegend' : False
})

## 유럽 확진자 선 scatter 트레이스 추가
fig_line.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x' : df_covid19_100_wide.index, 'y' : df_covid19_100_wide['확진자_유럽'],
    'name' : '유럽'
})

## 북미 확진자 선 scatter 트레이스 추가
fig_line.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x' : df_covid19_100_wide.index, 'y' : df_covid19_100_wide['확진자_북미'],
    'name' : '북미'
})

## 범례 layout 설정
fig_line.update_layout(
    title = dict(text = '대륙별 신규 확진자수 추이', x = 0.5, xanchor = 'center', yanchor = 'top'),
    legend = dict(orientation = 'v', bordercolor = 'gray', borderwidth = 2, x = 0.95, y = 0.95, xanchor = 'right'),
    showlegend = True)

fig_line.show()

In [ ]:
fig_line.update_layout(
    ## 여백 설정
    margin = dict(t = 50, b = 25, l = 25, r = 25)
)

fig_line.show()

In [ ]:
  ## 플롯 사이즈 설정
fig_scatter_temp.update_layout(width = 450, height = 700)

fig_scatter_temp.show()

In [ ]:
  ## 폰트 설정
fig_line.update_layout(font = dict(family = "나눔고딕", color = 'MidnightBlue', size = 15))

fig_line.show()

In [ ]:
###############################################################################################
## chap 5. 서브플롯
###############################################################################################

In [ ]:
## 서브플롯 생성을 위한 라이브러리 임포트
from plotly.subplots import make_subplots

## 서브플롯 생성을 위한 기본 Plotly 객체 생성
fig_subplot = make_subplots(
    rows=3, cols=3,
    ## 서브플롯 제목 설정
    subplot_titles = ('한국', '아시아', '유럽', '북미', '남미', '오세아니아', '아프리카'))

## 첫 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_한국'],
    'name':'한국'},
    row=1, col=1)

## 두 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아시아'],
    'name':'아시아'},
    row=1, col=2)

## 세 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_유럽'],
    'name':'유럽'},
    row=1, col=3)

## 네 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_북미'],
    'name':'북미'},
    row=2, col=1)

## 다섯 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_남미'],
    'name':'남미'},
    row=2, col=2)

## 여섯 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아프리카'],
    'name':'아프리카'},
    row=2, col=3)

## 일곱 번째 서브플롯 생성
fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_오세아니아'],
    'name':'오세아니아'},
    row=3, col=1)

fig_subplot.update_layout(title=dict(text = "최근 100일간 코로나19 확진자", 
                            x = 0.5))

fig_subplot.show()

In [ ]:
    ## 범례는 제거
fig_subplot.update_layout(showlegend = False)

fig_subplot.show()

In [ ]:
from plotly.subplots import make_subplots

fig_subplot = make_subplots(
    rows=3, cols=3,
    column_widths = [0.5,0.25,0.25], ## 열 방향 너비 설정
    row_heights = [0.5,0.25,0.25], ## 행 방향 너비 설정
    subplot_titles = ('한국', '아시아', '유럽', '', '북미', '남미', '', '오세아니아', '아프리카'))

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_한국'],
    'name':'한국'},
    row=1, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아시아'],
    'name':'아시아'},
    row=1, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_유럽'],
    'name':'유럽'},
    row=1, col=3)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_북미'],
    'name':'북미'},
    row=2, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_남미'],
    'name':'남미'},
    row=2, col=3)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아프리카'],
    'name':'아프리카'},
    row=3, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_오세아니아'],
    'name':'오세아니아'},
    row=3, col=3)

fig_subplot.update_layout(title=dict(text = "최근 100일간 코로나19 확진자",
                            x = 0.5))

fig_subplot.show()

In [ ]:
from plotly.subplots import make_subplots

fig_subplot = make_subplots(
    rows=4, cols=3,
    ## specs를 사용한 서브플롯 구성 설정
    specs = [
        [{'colspan' : 3, 'rowspan' : 2}, None, None],
        [None, None, None],
        [{}, {}, {}],
        [{}, {}, {}]
        ],
        subplot_titles = ('한국', '아시아', '유럽', '북미', '남미', '오세아니아', '아프리카'))

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_한국'],
    'name':'한국'},
    row=1, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아시아'],
    'name':'아시아'},
    row=3, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_유럽'],
    'name':'유럽'},
    row=3, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_북미'],
    'name':'북미'},
    row=3, col=3)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_남미'],
    'name':'남미'},
    row=4, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아프리카'],
    'name':'아프리카'},
    row=4, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_오세아니아'],
    'name':'오세아니아'},
    row=4, col=3)

fig_subplot.update_layout(title=dict(text = "최근 100일간 코로나19 확진자",
                            x = 0.5))

fig_subplot.show()

In [ ]:
fig_subplot = make_subplots(
    rows=4, cols=3,
    specs = [
        [{'colspan' : 3, 'rowspan' : 2}, None, None],
        [None, None, None],
        [{}, {}, {}],
        [{}, {}, {}]
    ],
    ## shareX, shareY를 TRUE로 설정하여 축 공유
    shared_xaxes = True, shared_yaxes = True,
    subplot_titles = ('한국', '아시아', '유럽', '북미', '남미', '오세아니아', '아프리카'))

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_한국'],
    'name':'한국'},
    row=1, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아시아'],
    'name':'아시아'},
    row=3, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_유럽'],
    'name':'유럽'},
    row=3, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_북미'],
    'name':'북미'},
    row=3, col=3)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_남미'],
    'name':'남미'},
    row=4, col=1)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_아프리카'],
    'name':'아프리카'},
    row=4, col=2)

fig_subplot.add_trace({
    'type' : 'scatter', 'mode' : 'lines',
    'x': df_covid19_100_wide.index, 'y': df_covid19_100_wide['확진자_오세아니아'],
    'name':'오세아니아'},
    row=4, col=3)

fig_subplot.update_layout(title=dict(text = "최근 100일간 코로나19 확진자",
                            x = 0.5))

fig_subplot.show()

In [ ]:
###############################################################################################
## chap 6. 색상 설정
###############################################################################################

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    'marker' : {'color' : df_취업률_500['취업률'], 'colorscale' : 'Blues'}})

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace({
    'type' : 'scatter', 'mode' : 'markers',
    'x': df_취업률_500['졸업자수'], 'y': df_취업률_500['취업자수'],
    'marker' : {'color' : df_취업률_500['취업률'], 'colorscale' : 'Blues',
                'cmin' : 50, 'cmax' : 100,
                'colorbar' : {'title' : '취업률', 'ticksuffix' : '%'},
                'reversescale' : True}})

fig.show()

In [ ]:
fig = go.Figure()

## 이산형 변수를 수치형 변수로 변환하는 함수
def cat(row):
    key = row['location']
    value = {'한국': 0 , '아시아' : 1, '유럽' : 2, '북미' : 3, '남미' : 4, '아프리카' : 5, #
             '오세아니아' : 6}.get(key)
    return value

## 시각화를 위한 데이터 전처리
temp = df_covid19_100.groupby('location').agg(new_cases = ('new_cases', 'sum')).reset_index()

## 이산형 변수 열을 사용해 수치형 변수열 생성
temp['loc_code']= temp.apply(cat, axis=1)

fig.add_trace({
    'type' : 'bar', ## bar 트레이스 설정
    'x': temp['location'], 'y': temp['new_cases'],
    ## 이산형 변수를 변환한 수치형 변수를 color로 설정
    'marker' : {'color' : temp['loc_code'], 'colorscale' : 'Blues'}, 'showlegend' : True
})

fig.show()

In [ ]:
fig = go.Figure()

## 이산형 변수와 색상을 매칭하는 딕셔너리 정의
colors = {'한국': 'blue' , '아시아' : 'gray', '유럽' : 'gray', '북미' : 'gray',
'남미' : 'gray', '아프리카' : 'gray', '오세아니아' : 'gray'}
temp = df_covid19_100.groupby('location').agg(new_cases = ('new_cases', 'sum')).reset_index()

## 'location'의 그룹에 따라 for 루프
for cat, group in temp.groupby('location'):
    fig.add_trace({
        'type' : 'bar', ## bar 트레이스 설정
        'x': group['location'], 'y': group['new_cases'],
        'name' : cat,
        'marker' : {'color' : colors[cat]}
    })
    
fig.show()

In [ ]:
fig = go.Figure()

temp = df_covid19_100.groupby('location').agg(new_cases = ('new_cases', 'sum')).reset_index()

for cat, group in temp.groupby('location'):
    fig.add_trace({
        'type' : 'bar', ## bar 트레이스 설정
        'x': group['location'], 'y': group['new_cases'],
        'name' : cat
    })

fig.update_layout({'colorway' : ('blue', 'gray', 'gray', 'gray', 'gray', 'gray', 'gray')})

fig.show()

In [ ]:
import plotly.express as px

print(px.colors.sequential.Viridis)

['#440154', '#482878', '#3e4989', '#31688e', '#26828e', '#1f9e89', '#35b779', '#6ece58', '#b5de2b', '#fde725']